In [1]:
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.models import Sequential
# # from mne.decoding import SPoC
# import tensorflow.keras.backend as K
# from keras import Input ,Model
# from keras.layers import RepeatVector, Permute ,Activation
# from tensorflow.keras.backend import squeeze, dot, expand_dims,tanh, exp
# from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
# from tensorflow.keras.layers import Layer
# from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Flatten, Conv2D, MaxPooling2D, Conv1D,MaxPool2D,GRU, Dropout, MaxPool1D, ConvLSTM2D,MaxPooling1D, ReLU, LeakyReLU, RNN, SimpleRNN, GRU, LSTM, TimeDistributed
# from tensorflow.keras.losses import sparse_categorical_crossentropy
# from tensorflow.keras.optimizers import Adam, SGD
# from tensorflow.keras.layers import BatchNormalization
# from sklearn.model_selection import KFold
# from scipy.interpolate import interp1d
# import tensorflow as tf
# from scipy.signal import stft, spectrogram
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import h5py # Read and write HDF5 files from Python
# import os
# import matplotlib.pyplot as plt
# import matplotlib
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from sklearn.metrics import confusion_matrix
# from keras import Input ,Model
# from tensorflow.keras.layers import concatenate
import h5py  # Read and write HDF5 files from Python
import matplotlib
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras import Input, Model
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.signal import stft, spectrogram
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Lambda, GlobalMaxPool1D,LSTM,BatchNormalization,MaxPooling1D, Dense, Flatten, Conv1D, Dropout, GRU, TimeDistributed, LeakyReLU, MaxPool1D, GlobalAveragePooling1D
# from mne.decoding import SPoCMaxPooling1D,
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import concatenate
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import scipy.io as sio
# from hyperas.distributions import choice, uniform
data_path = r"C:\Users\eidan\Documents\BME_Dreem\new"
file_xtrain = data_path + r"/X_train.h5"
file_xtest = data_path + r"/X_test.h5"
file_ytrain = data_path + r"/y_train.csv"

In [2]:


# # training labels
# pd.read_csv(file_ytrain)
# # # what does the h5 file contains ?
# # with h5py.File(file_xtrain, "r") as hf:
# #        print(list(hf.keys()))

# # # How to load data from h5? what is its shape and type?
# # with h5py.File(file_xtrain, "r") as hf:
# #        field = list(hf.keys())[0]
# #        x_data = hf[field][()]
# # type(x_data), x_data.shape


# ############################################################################################################################
import warnings

import numpy as np
import scipy.fftpack as fft
import scipy
import scipy.signal
import scipy.interpolate
import six

def power_to_db(S, ref=1.0, amin=1e-10, top_db=80.0):
    """Convert a power spectrogram (amplitude squared) to decibel (dB) units"""
    
    S = np.asarray(S)

    if amin <= 0:
        raise ParameterError('amin must be strictly positive')

    if np.issubdtype(S.dtype, np.complexfloating):
        warnings.warn('power_to_db was called on complex input so phase '
                      'information will be discarded. To suppress this warning, '
                      'call power_to_db(magphase(D, power=2)[0]) instead.')
        magnitude = np.abs(S)
    else:
        magnitude = S

    if six.callable(ref):
        # User supplied a function to calculate reference power
        ref_value = ref(magnitude)
    else:
        ref_value = np.abs(ref)

    log_spec = 10.0 * np.log10(np.maximum(amin, magnitude))
    log_spec -= 10.0 * np.log10(np.maximum(amin, ref_value))

    if top_db is not None:
        if top_db < 0:
            raise ParameterError('top_db must be non-negative')
        log_spec = np.maximum(log_spec, log_spec.max() - top_db)

    return log_spec

# #####################################################################################################################

def normalize_data(eeg_array):
    """normalize signal between 0 and 1"""
    sort = -np.sort(-np.abs(eeg_array), axis=1)
    sort_mean = np.mean(sort[:, 0:3, :], axis=1)
    normalized_array1 = np.copy(eeg_array)
    for i in range(0,eeg_array.shape[0]):
        for j in range(0,eeg_array.shape[2]):
            normalized_array1[i,:,j] = eeg_array[i,:,j] / sort_mean[i,j]
    normalized_array = np.clip(eeg_array, -150, 150)
    normalized_array = normalized_array / 150

    return normalized_array1


def stft_preprocessing(data, mean= None, var=None):
    """Transform the signal in input in his STFT version, add a dimension"""
    
    shapedata = data.shape
    
#     newdata = np.zeros((shapedata[0],shapedata[-1], 48, 65,))
    
#     newdata = np.zeros((shapedata[0],shapedata[-1], 13, 129,))
    
#     for i in range(shapedata[-1]):
#     _, _, Zxx  = stft(data ,fs = 50, axis= 1)
    Zxx = stft(data, fs=50, nperseg=128, nfft=128, noverlap=64, axis=1)[2]
    print(Zxx.shape)
#         _, _, Zxx=  stft(data[:,:,i] ,nperseg=128,nfft=128,fs = 50, axis= 1)
#     Zxx = power_to_db(Zxx, ref = np.mean)
    Zxx = np.log(np.abs(Zxx))
    cliped = np.clip(Zxx,-20,20)
    cliped = np.swapaxes(cliped,1,2)
    cliped = np.swapaxes(cliped,2,3)
#         newdata[:,i,:,:] = cliped
    
    
    newdata = cliped
    print(newdata.shape)
    if mean ==None: 
        mean = newdata.mean() 
        

    newdata = newdata-mean
  
    if var==None :
        var = newdata.var()
    
    newdata = newdata/var

        
    
    return (newdata, mean,var)



#----------------------------------------------------------------------------------------------------

def SPoC_preprocessing(x_train, y_train, x_validation, n_components= 6):

    x_train = np.moveaxis(x_train,1,-1)
    spoc_train= SPoC(n_components=n_components, reg='oas', rank='full', transform_into='csp_space')
    spoc_train.fit(x_train,y_train)

    x_train_copy = spoc_train.transform(x_train)
    x_train_copy = np.moveaxis(x_train_copy,1,-1)

    x_validation_copy = np.moveaxis(x_validation,1,-1)
    x_validation_copy = spoc_train.transform(x_validation_copy)
    x_validation_copy = np.moveaxis(x_validation_copy,1,-1)
    
    return (x_train_copy,x_validation_copy, spoc_train)


 #----------------------------------------------------------------------------------------------------

def coordinate_preprocessing(data, mean = None , var = None):
    
    
    if mean is None: 
        mean =  data.mean(axis=(0,1))
    else :
        data = data-mean
  
    if var is None :
        var = data.var(axis= (0,1))

    else :
        data = data/var
    
    
    data = np.diff(data, axis = 1)
    return (data , mean , var)
    
#----------------------------------------------------------------------------------------------------
def positional_embeding(data): 
    
    data = np.array(data)
    
    pos = np.zeros([data.shape[0],6])
    
    pos[:,0]= data/1200
    
    
    angle =  [30, 60, 90, 120, 150] 
    for i in range(5):
        pos[:,i+1]=np.cos((data * np.pi)/angle[i]) 
    
    
    return pos 
        
    
    
def split_data(input_signals_list,validation_ratio=0.3):
    with h5py.File(file_xtrain, "r") as fi:
        if len(input_signals_list) == 1:
            x_data = fi[input_signals_list[0]][()]
        else:
            x_data = np.zeros([24688,1500,len(input_signals_list)], dtype=np.float64)
            for i in range(0, len(input_signals_list)):
                if 'x' in input_signals_list[i] or 'y' in input_signals_list[i] or 'z' in input_signals_list[i]:
                    f1 = interp1d(np.arange(0, 300), fi[input_signals_list[i]][()], axis=1)
                    xnew = np.linspace(0, 30, num=1500)
                    x_data[0:24688, 0:1500, i] = f1(xnew)
                else:
                    x_data[0:24688, 0:1500, i] = fi[input_signals_list[i]][()]
        
        y_data = pd.read_csv(file_ytrain)['sleep_stage'].to_numpy()
        metadata =  fi["index_window"][()]
        
        
        # Converting to 1 minutes epochs
#         x_data = np.append(x_data[0:-1,:,:],x_data[1:,:,:],axis=1)
#         y_data = y_data[1:]
        
#         metadata=np.delete(metadata,0)
#         n1 = np.where(y_data == 1)[0]
#         x_data_n1 = x_data[n1,:,:]
#         for i in [0,3,5]:
#             x_data_n1 = x_data[n1, :, :]
#             x_data_n1[:,:,i] = 0
#             x_data = np.append(x_data, x_data_n1, axis=0)
#         # EEG extraction
#         y_data=np.append(y_data,np.ones(np.abs(x_data.shape[0]-y_data.shape[0])))
#         metadata = np.concatenate([metadata,metadata[n1],metadata[n1],metadata[n1]])
        mask_eeg = [0,1,2,3]
        
        # You can't do spoc with K fold, or at least not here
        # x_train_spoc, x_validation_spoc, spoc = SPoC_preprocessing(x_train[:,:,mask_eeg], y_train, x_validation[:,:,mask_eeg], 4)

        x_eeg_normalized = normalize_data(x_data[:,:,mask_eeg]) 
        x_eeg = np.copy(x_eeg_normalized)
#         x_eeg = np.append(x_eeg,x_eeg,axis=0)
        print('kill them allllll')
        for k in range(0,x_eeg.shape[0]):
            for l in range(0,x_eeg.shape[2]):
                if np.random.uniform() < 0.1:
                    x_eeg[k,:,l] = x_eeg[k,:,l] * 0
        x_eeg_normalized = np.copy(x_eeg)
        print('JK just 50% of them')
        print('ok 40% ...')
#         x_eeg_normalized = np.append(x_eeg_normalized[0:-1,:,:],x_eeg_normalized[1:,:,:],axis=1)
#         n1 = np.where(y_data==1)[0]
#         x_eeg_normalized = np.append(x_eeg_normalized,x_eeg_normalized,axis=0)
        
#         y_data = np.append(y_data,y_data,axis=0)
#         x_eeg_normalized = np.append(x_eeg_normalized,x_eeg_normalized[n1],axis=0)
        
#         y_data = np.append(y_data,y_data[n1],axis=0)
#         print('strat augmntetion of data')
#         for k in range(0,x_eeg_normalized.shape[0]):
#             for l in range(0,x_eeg_normalized.shape[2]):
#                 if np.random.uniform() < 0.2:
#                     x_eeg_normalized[k,:,l] = x_eeg_normalized[k,:,l] * 0
#         print('finish augmntetion of data')
        x_eeg ,mean_eeg, var_eeg = stft_preprocessing(x_eeg)
        
        # X Y Z
        
        if 'x' in input_signals_list :
            mask_mvt= np.argwhere(np.logical_or(np.logical_or(input_signals_list == "x", input_signals_list == "y" ) , input_signals_list == "z" ))
#             print(mask_mvt)
            x_mvt, mean_mvt , var_mvt = coordinate_preprocessing(x_data[:,:,mask_mvt])
            x_mvt = x_data[:,:,mask_mvt]
#             print(x_mvt.shape)
            x_mvt = np.squeeze(x_mvt)
#             print(x_mvt.shape)

        
        # index
        x_meta = positional_embeding(metadata)
        x_meta = np.append(x_meta[0:-1,:],x_meta[1:,:],axis=1)
        print(x_meta.shape)
        x_meta=x_meta.reshape(24687,2,6)
        x_eeg = np.append(x_eeg[0:-1,:,:,:],x_eeg[1:,:,:,:],axis=1)
        print(x_eeg.shape)
        # x_data.shape
        x_eeg = x_eeg.reshape(24687,2,4,25,65)
        
#         print(x_eeg_normalized.shape)
        x_eeg_normalized = np.append(x_eeg_normalized[0:-1,:,:],x_eeg_normalized[1:,:,:],axis=1)
        # x_data.shape
        y_data = y_data[1:]
#         x_eeg_normalized = x_eeg_normalized.reshape(24688*2-1,3000,7)
        
        
#         x_meta = np.append(x_meta[0:-1,:],x_meta[1:,:],axis=1)
        print(x_meta.shape)
#         x_meta=x_meta.reshape(24688*2-1,2,6)
#         y_data = np.append(y_data[1:],y_data)
#         x_meta=x_meta[1:,]
        
        #x_data_to_connect = x_data[1:,:,:]
#         x_data = np.concatenate((x_data[0:-2,:,:],x_data[1:-1,:,:],x_data[2:,:,:]),axis=1)
#         for a in range(0,x_data.shape[0]-1):
#             x_data_new=np.zeros([x_data.shape[0]-1,3000,x_data.shape[2]])
#             x_data_new[a,:,:] = np.append(x_data[a,:,:],x_data[a+1,:,:],axis=0)
#         x_data = x_data_new
#         x_data_new = 0

#        x_metadata = x_metadata[1:]
#        print(x_metadata.shape)
#        y_data =np.delete(y_data,0)
        # Creating data indices for training and validation splits:
#        dataset_size = len(y_data)
#        indices = list(range(dataset_size))
#        split = int((1 - validation_ratio) * dataset_size)
#        np.random.shuffle(indices)

        
        
#        train_indices, val_indices = np.argwhere(x_metadata%5 != 0) ,np.argwhere(x_metadata%5 == 0)
#        train_indices, val_indices =  np.squeeze(train_indices), np.squeeze(val_indices)


#        x_train, x_validation = x_data[train_indices], x_data[val_indices]
#        y_train, y_validation = y_data[train_indices], y_data[val_indices]

        print('data ready')
    return x_eeg_normalized,x_eeg, x_meta,y_data
# input_signals_list = ['eeg_1','eeg_2','eeg_3','eeg_4','eeg_5','y']
#input_signals_list = ['eeg_3','eeg_4','eeg_5']
#input_signals_list = ['eeg_1','eeg_2','eeg_3','eeg_4','eeg_5','eeg_6','eeg_7']
input_signals_list = ['eeg_4','eeg_5', 'eeg_6', 'eeg_7','x','y','z']
# input_signals_list = ['eeg_4','eeg_1']
# input_signals_list = [ 'eeg_3', 'eeg_4', 'eeg_5']
# input_signals_list = np.array(['eeg_3','eeg_4','eeg_5', 'eeg_6', 'eeg_7',"x", "y","z"])
# input_signals_list = np.array(['eeg_2','eeg_3','eeg_4','eeg_5', 'eeg_6',"x", "y","z"])

x_eeg_normalized, x_eeg, x_meta,y_data = split_data(input_signals_list)

kill them allllll
JK just 50% of them
ok 40% ...
(24688, 65, 4, 25)


<ipython-input-2-99cc04459482>:87: RuntimeWarning: divide by zero encountered in log
  Zxx = np.log(np.abs(Zxx))


(24688, 4, 25, 65)
(24687, 12)
(24687, 8, 25, 65)
(24687, 2, 6)
data ready


In [3]:
# # print(x_mvt.shape)
print('y_data.shape = ',y_data.shape)
print('x_eeg.shape = ',x_eeg.shape)
print('x_eeg_normalized.shape = ',x_eeg_normalized.shape)
print('x_meta.shape = ',x_meta.shape)
# x_eeg = np.swapaxes(x_eeg,2,3)
# print('x_eeg.shape = ',x_eeg.shape)
# # # n1 = np.where(y_data==1)[0]
# # x_eeg_normalized = np.append(x_eeg_normalized,x_eeg_normalized,axis=0)
# # x_eeg = np.append(x_eeg,x_eeg,axis=0)
# # x_meta = np.append(x_meta,x_meta,axis=0)
# # y_data = np.append(y_data,y_data,axis=0)
# # x_eeg_normalized = np.append(x_eeg_normalized,x_eeg_normalized[n1],axis=0)
# # x_eeg = np.append(x_eeg,x_eeg[n1],axis=0)
# # x_meta = np.append(x_meta,x_meta[n1],axis=0)
# # y_data = np.append(y_data,y_data[n1],axis=0)
# # print(y_data.shape)
# # print(x_eeg.shape)
# # print(x_eeg_normalized.shape)
# # print(x_meta.shape)

# # x_eeg = np.append(x_eeg[0:-1,:,:,:],x_eeg[1:,:,:,:],axis=1)
# # x_data.shape
# # x_eeg = x_eeg.reshape(24687,2,7,65,25)
# mdic = {"x_eeg_normalized": x_eeg_normalized,"x_eeg": x_eeg,"x_meta": x_meta,"y_data":y_data}
# sio.savemat("to_load4_data_aug.mat", mdic)
# print('y_data.shape = ',y_data.shape)
# print('x_eeg.shape = ',x_eeg.shape)
# print('x_eeg_normalized.shape = ',x_eeg_normalized.shape)
# print('x_meta.shape = ',x_meta.shape)

y_data.shape =  (24687,)
x_eeg.shape =  (24687, 2, 4, 25, 65)
x_eeg_normalized.shape =  (24687, 3000, 4)
x_meta.shape =  (24687, 2, 6)


In [4]:
# # x_eeg_normalized = sio.loadmat("to_load3.mat",matlab_compatible=True)['x_eeg_normalized']
# x_eeg = sio.loadmat("to_load4.mat",matlab_compatible=True)['x_eeg']
# x_meta = sio.loadmat("to_load4.mat",matlab_compatible=True)['x_meta']
# y_data = sio.loadmat("to_load4.mat",matlab_compatible=True)['y_data']
# y_data = y_data.T
# x_eeg = np.append(x_eeg[0:-1,:,:,:],x_eeg[1:,:,:,:],axis=1)
# # x_data.shape
# x_eeg = x_eeg.reshape(24687,2,7,25,65)

# x_meta = np.append(x_meta[0:-1,:],x_meta[1:,:],axis=1)
# print(x_meta.shape)
# x_meta=x_meta.reshape(24687,2,6)


# print('y_data.shape = ',y_data.shape)
# print('x_eeg.shape = ',x_eeg.shape)
# # print('x_eeg_normalized.shape = ',x_eeg_normalized.shape)
# print('x_meta.shape = ',x_meta.shape)


In [5]:
# occurence = np.zeros((5))
# for i in range(5):
#     occurence[i]= np.sum(np.where(y_data == i))
# #     print(occurence)
# # plt.bar([1,2,3,4,5] , occurence )
# weight =occurence/sum(occurence)
# #     print(1- weight)
# weight = - np.log( occurence/sum(occurence))
# print(weight)

In [6]:
optimizer = Adam(learning_rate=0.001)

def custom_loss(ytrue, ypred):
    
    
    scce = tf.keras.losses.SparseCategoricalCrossentropy()
    
#     weight =tf.constant([[0.85347313, 0.93098802, 0.6152054,  0.79114608, 0.80918737]])
    #weight =tf.constant([[0.00001, 1.0000, 0.00001, 0.00001, 0.001]])
    weight = tf.constant([[0.94355903, 2.70683738, 0.96141876, 0.55233263, 0.62986756]])
#     weight = tf.constant([[1.92961025, 2.68815953, 0.96311429, 1.17516681, 1.21891314]])
#     weight = tf.constant([[2.00664824, 2.05667881, 1.02830803, 1.63059645, 1.69141161]])
    
    new_y = tf.expand_dims(ypred, axis=1)


    new_weight = tf.matmul(weight,new_y, transpose_b = True)
    
    score = scce(ytrue,ypred, sample_weight= new_weight)
    
    return score

In [7]:
# # Define the K-fold Cross Validator
kfold = KFold(n_splits=100, shuffle=True)
# kfold = KFold(n_splits=40, shuffle=False)
# # K-fold Cross Validation model evaluation
fold_no = 1

In [8]:
class Attention(Layer):

    def __init__(self,input_dim, context_size=25, activation='tanh'):

        super(Attention,self).__init__() # constructor of Base Layer class
        init = tf.initializers.GlorotUniform()
        self.context_matrix = tf.Variable(init(shape=(context_size,input_dim)),trainable=True)
        self.context_bias = tf.Variable(init(shape=(1,context_size)),trainable=True)
        self.context_vector = tf.Variable(init(shape=(context_size,1)),trainable=True)
    def get_config(self):

        config = super().get_config()
        config.update({
            'context_matrix': self.context_matrix,
            'context_bias': self.context_bias,
            'context_vector': self.context_vector
        })
        return config
    def call(self,x):
        """
        x (tensor: batch_size,sequence length,input_dim):
        returns x (tensor: batch_size,input_dim),  attention_weights (tensor: batch_size,sequence_length)
        """
        batch_size, length, n_features = x.shape
        x_att = tf.reshape(x,[-1, n_features])
        u = tf.linalg.matmul(x_att,tf.transpose(self.context_matrix,perm=[1,0])) + self.context_bias
        u = tf.nn.tanh(u)
        uv = tf.linalg.matmul(u,self.context_vector)
        uv = tf.reshape(uv, [-1, length])
        alpha = tf.nn.softmax(uv,axis=1)
        alpha = tf.expand_dims(alpha, axis=-1)
        x_out = alpha * x
        x_out=tf.math.reduce_sum(x_out, axis=1)
#         if np.random.uniform() < 0.2:
#             x_out = x_out * 0
#         if np.random.uniform() < 0.7:
#             x_out = x_out * 0


        return x_out

In [9]:
def custom_Kfold(n): 
    
    with h5py.File(file_xtrain, "r") as fi:
        x_metadata = fi['index'][()]

    x_metadata = x_metadata[1:]
#     x_metadata = np.append(x_metadata[1:],x_metadata,axis=0)
    indice = []
    
    for i in range(n): 
        train_indices, val_indices = np.argwhere(x_metadata%n != 0+i) ,np.argwhere(x_metadata%n == 0+i)
        np.random.shuffle(train_indices)
        np.random.shuffle(val_indices)
        indice.append(  {"train":np.squeeze(train_indices), "validation": np.squeeze(val_indices)})
    
    return(indice)
dic_kfold = custom_Kfold(5)
def backend_reshape(x):
    return tf.keras.backend.reshape(x, (-1, n_channels, spectrogram_length, filter_size))

In [10]:

# units = 128
hidden=None
C=4
att_context=20
filter_reduc=40
m1=128
m2=128
p1=0.2
p2=0.2
time_step =48 # Or 48
fold_no=1
_,temporal_context,n_channels, spectrogram_length, filter_size = x_eeg.shape
number_channels = n_channels
inputA = Input(shape=(temporal_context,n_channels, spectrogram_length, filter_size))
#     inputB = Input(shape=(1499,3))
# inputB = Input(shape=(2999,3))
inputC = Input(shape=(2,6))
inputD = Input(shape=(134))
inputH = Input(shape=(3000,number_channels))




#  Channel redudcution
k= Lambda(backend_reshape)(inputA)
# k= tf.transpose(k,perm=[0,1,3,2])
k= TimeDistributed(Dense(filter_reduc))(k)
k= tf.transpose(k,perm=[0,3,2,1])
k= TimeDistributed(Dense(C))(k)
k= tf.transpose(k,perm=[0,3,2,1])
k= tf.keras.layers.Dropout(p1)(k)

k= tf.keras.layers.Reshape((-1, spectrogram_length))(k)
k= tf.transpose(k,perm=[0,2,1])

# GRU
k = tf.keras.layers.Bidirectional(GRU(m1 ,return_sequences=True, go_backwards=False))(k)
k= tf.keras.layers.Dropout(p1)(k)
# k = tf.keras.layers.Bidirectional(GRU(units ,return_sequences=True, go_backwards=False))(k)
# k= tf.transpose(k,perm=[0,2,1])
# k= tf.keras.layers.Reshape((-1,m1))(k)
k=Attention(m1*2,att_context)(k)
# print(k.shape)
k = tf.reshape(k, [-1,temporal_context, 2*m1])
# k=Dropout(0.1)(k)
model_A = Model(inputs=inputA, outputs= k )
#     print(model_A.summary())
# print(k.summary())


# (B,context,2m+6)
# m = Dense(24, activation=tf.nn.relu)(inputC)
m = Dense(6, activation=tf.nn.relu)(inputC)
m = Dense(6, activation=tf.nn.relu)(m)
m = Dense(6, activation=tf.nn.relu)(m)
m = Dense(6, activation=tf.nn.relu)(m)
m = Dense(6, activation=tf.nn.relu)(m)
model_C = Model(inputs=inputC, outputs=m)


combined_input = concatenate([model_A.output, model_C.output])

#     print('k',combined_input.shape)
combined=tf.keras.layers.Reshape((combined_input.shape[1],-1))(combined_input)
#     print('k',combined.shape)
# combined = tf.transpose(combined,perm=[0,2,1])
x_residual = Dense(m1*2)(combined)
# combined = tf.transpose(combined,perm=[0,2,1])
x_lstm = tf.keras.layers.Bidirectional(GRU(m2,return_state=True,return_sequences=True))(combined,initial_state=hidden)#1
hidden=x_lstm[1:]
x_lstm=x_lstm[0]
# x_lstm = tf.transpose(x_lstm,perm=[0,2,1])
x_cat = (x_residual + x_lstm) / 2
combined = Dropout(p2)(x_cat)
x_residual = Dense(m1*2)(combined)
# combined = tf.transpose(combined,perm=[0,2,1])
x_lstm = tf.keras.layers.Bidirectional(GRU(m2,return_state=True,return_sequences=True))(combined,initial_state=hidden)#2
hidden=x_lstm[1:]
x_lstm=x_lstm[0]
# x_lstm = tf.transpose(x_lstm,perm=[0,2,1])
x_cat = (x_residual + x_lstm) / 2
combined = Dropout(p2)(x_cat)
combined = GlobalAveragePooling1D()(combined)
# combined = Flatten()(combined)
# combined = Dense(m1*2, activation=tf.nn.leaky_relu)(combined)
# v = Dense(5, activation="softmax")(combined)
combined = Model(inputs = [model_A.input, model_C.input] , outputs= combined)
# model1 = Model(inputs = [model_A.input, model_C.input] , outputs= v)




# y = Conv1D(3,10, strides=10,activation=tf.nn.leaky_relu)(inputB)
# y = Conv1D(2,5, strides =5, activation=tf.nn.leaky_relu)(y)
# y = Dropout(0.25)(y)
# y = Conv1D(1,2 ,strides=2 ,activation=tf.nn.leaky_relu)(y)
# y = Dropout(0.25)(y)
# y = Flatten()(y)
# """
# y = Dense(100, activation=tf.nn.leaky_relu)(y)
# y= Dense(50, activation=tf.nn.leaky_relu)(y)
# y= Dense(5, activation="linear")(y)"""

# y = Model(inputs=inputB, outputs=y)




raws = BatchNormalization()(inputH)
raws = Conv1D(filters=60, kernel_size=20, padding='SAME', activation=tf.nn.leaky_relu)(raws)
raws = MaxPooling1D(pool_size=6)(raws)
raws = Conv1D(filters=80, kernel_size= 15, padding='SAME', activation=tf.nn.leaky_relu)(raws)
raws = MaxPooling1D(pool_size=6)(raws)
raws = Dropout(0.25)(raws)
raws = Conv1D(filters=80, kernel_size= 10, padding='SAME', activation=tf.nn.leaky_relu)(raws)
raws = MaxPooling1D(pool_size=6)(raws)
raws = Conv1D(filters=80, kernel_size= 10, padding='SAME', activation=tf.nn.leaky_relu)(raws)
raws = MaxPooling1D(pool_size=4)(raws)
raws = Dense(128, activation=tf.nn.leaky_relu)(raws)
z = GlobalAveragePooling1D()(raws)
    
# z = Conv1D(128, kernel_size=7, strides=2, padding='SAME')(inputH)
# z = LeakyReLU(alpha=0.1)(z)
# z = Conv1D(128, kernel_size=7, strides=2, padding='SAME')(z)
# z = LeakyReLU(alpha=0.1)(z)
# z = Dropout(0.25)(z)
# z = Conv1D(128, kernel_size=7, strides=2, padding='SAME')(z)
# z = LeakyReLU(alpha=0.1)(z)
# z = MaxPool1D(4)(z)
# z = Conv1D(256, kernel_size=7, strides=2, padding='SAME')(z)
# z = Conv1D(128, kernel_size=7, strides=2, padding='SAME')(z)
# z = LeakyReLU(alpha=0.1)(z)
# z = Flatten()(z)
z = Model(inputs=inputH, outputs=z)

last_layer =  concatenate([combined.output,z.output])
#     z = Dense(200, activation=tf.nn.leaky_relu)(last_layer)

#     z = Dense(150, activation=tf.nn.leaky_relu)(z)
#     z= Dropout(0.25)(z)
# v = Dense(25, activation=tf.nn.leaky_relu)(last_layer)
v = Dense(5, activation="softmax")(last_layer)

model1 = Model(inputs=[model_A.input, model_C.input,z.input], outputs=v)
print(model1.summary())
# model2 = Model(inputs=[model_A.input, y.input, model_C.input], outputs=z)
# model3 = Model(inputs=[model_A.input, y.input, model_C.input], outputs=z)


# #######################################################################
# # both the temporal and epoch needs to be reshape as follow and Concatenate over the -1 axis
# # to have (batch_size, k.shape[1]+6,2) shape.
#     print('k',k.shape)
#     k=tf.keras.layers.Reshape((k.shape[1],-1))(k)
#     print('k',k.shape)
# #     k = tf.transpose(k,perm=[0,2,1])
# ################################################################
# #2-SkipGRU
#     x_residual = Dense(units*2)(k)
# #     k = tf.transpose(k,perm=[0,2,1])
#     x_lstm = tf.keras.layers.Bidirectional(GRU(units,return_state=True,return_sequences=True))(k,initial_state=hidden)#1
#     hidden=x_lstm[1:]
#     x_lstm=x_lstm[0]
# #     x_lstm = tf.transpose(x_lstm,perm=[0,2,1])
#     x_cat = (x_residual + x_lstm) / 2
#     k = Dropout(0.25)(x_cat)
#     x_residual = Dense(units*2)(k)
# #     k = tf.transpose(k,perm=[0,2,1])
#     x_lstm = tf.keras.layers.Bidirectional(GRU(units,return_state=True,return_sequences=True))(k,initial_state=hidden)#2
#     hidden=x_lstm[1:]
#     x_lstm=x_lstm[0]
# #     x_lstm = tf.transpose(x_lstm,perm=[0,2,1])
#     x_cat = (x_residual + x_lstm) / 2
#     k = Dropout(0.25)(x_cat)
#     k = Flatten()(k)
#     k= Dense(100,activation=tf.nn.leaky_relu)(k)
#     k= Dense(5, activation='softmax')(k)

#     model.summary()


#model = Model(inputs=inputA, outputs=k)
#ompile the model

NotImplementedError: Cannot convert a symbolic Tensor (bidirectional/forward_gru/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
dic_kfold = custom_Kfold(30)
# 
for dic in dic_kfold: 
    train , test = dic.values()
# for train, test in kfold.split(x_eeg, y_data):
    if fold_no == 16:
        print(test.shape)
        print(f'Training for fold {fold_no} ...')
        model1.compile(loss=custom_loss,
                    optimizer=optimizer,
                    metrics=['accuracy'])
#         history = model1.fit([x_eeg, x_meta,x_eeg_normalized], y_data,batch_size=8,
#                   epochs=50,verbose=1)
        history = model1.fit([x_eeg[train], x_meta[train],x_eeg_normalized[train]], y_data[train],
                 batch_size=32, 
                             validation_data=([x_eeg[test], x_meta[test],x_eeg_normalized[test]], y_data[test])
                             ,callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience = 25)],
                  epochs=200,verbose=1)
        print('------------------------------------------------------------------------')
        print(f'DONE training for fold {fold_no} ...')
#     # Increase fold number
    fold_no = fold_no + 1

In [ ]:
a= pd.read_

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import winsound
winsound.Beep(1000, 900)

In [ ]:
# from keras.models import load_model
# model = load_model('./model.h5',compile=True)
# del x_train
# del x_validation

def test_data(input_signals_list):
    with h5py.File(file_xtest, "r") as fi:
        index = fi['index_absolute'][()]
        metadata =  fi["index_window"][()]
        if len(input_signals_list) == 1:
            x_data = fi[input_signals_list[0]][()]
        else:
            x_data = np.zeros([24980,1500,len(input_signals_list)], dtype=np.float64)
            for i in range(0, len(input_signals_list)):
                if 'x' in input_signals_list[i] or 'y' in input_signals_list[i] or 'z' in input_signals_list[i]:
                    f1 = interp1d(np.arange(0, 300), fi[input_signals_list[i]][()], axis=1)
                    xnew = np.linspace(0, 30, num=1500)
                    x_data[0:24980, 0:1500, i] = f1(xnew)
                else:
                    x_data[0:24980, 0:1500, i] = fi[input_signals_list[i]][()]
        

    # EEG extraction
    mask_eeg = [0,1,2,3]

    # You can't do spoc with K fold, or at least not here
    # x_train_spoc, x_validation_spoc, spoc = SPoC_preprocessing(x_train[:,:,mask_eeg], y_train, x_validation[:,:,mask_eeg], 4)

#     x_meta = positional_embeding(metadata)
    x_eeg_normalized = normalize_data(x_data[:,:,mask_eeg])
    x_eeg = np.copy(x_eeg_normalized)
    x_eeg ,mean_eeg, var_eeg = stft_preprocessing(x_eeg)
    x_meta = positional_embeding(metadata)
    x_meta = np.append(x_meta[0:-1,:],x_meta[1:,:],axis=1)
    print(x_meta.shape)
    x_meta=x_meta.reshape(24979,2,6)
    print(x_meta.shape)
    x_eeg = np.append(x_eeg[0:-1,:,:,:],x_eeg[1:,:,:,:],axis=1)
    print(x_eeg.shape)
    # x_data.shape
    x_eeg = x_eeg.reshape(24979,2,4,25,65)
    print(x_eeg.shape)
#         print(x_eeg_normalized.shape)
    x_eeg_normalized = np.append(x_eeg_normalized[0:-1,:,:],x_eeg_normalized[1:,:,:],axis=1)
    print(x_eeg_normalized.shape)

    # X Y Z

    if 'x' in input_signals_list :
        mask_mvt= np.argwhere(np.logical_or(np.logical_or(input_signals_list == "x", input_signals_list == "y" ) , input_signals_list == "z" ))
#             print(mask_mvt)
        x_mvt, mean_mvt , var_mvt = coordinate_preprocessing(x_data[:,:,mask_mvt])
#             print(x_mvt.shape)
        x_mvt = np.squeeze(x_mvt)
#             print(x_mvt.shape)


    # index
#     x_meta = positional_embeding(metadata)

    print('data ready')

    return x_eeg_normalized,x_eeg, x_meta ,index
x_eeg_normalized_test,x_eeg_test, x_meta_test ,index = test_data(input_signals_list)

y_test = model1.predict([x_eeg_test, x_meta_test,x_eeg_normalized_test])
y_test = np.argmax(y_test,axis=1)
y_test=np.append(y_test[0],y_test)
if (index.shape) == (y_test.shape):
    df = pd.DataFrame(data={'index': index, 'sleep_stage': y_test})
    df.to_csv('model_num_'+str(fold_no)+'_our_result_SkipGRU_2nd.csv',index=False)
    print('you can save the results')
else:
    print('there is an error in the shape of y_test')
# for fold_no in range(1,4):
#     model.load_weights("model_"+str(fold_no)+".hdf5")
#     y_test = model.predict_classes(Zxx_test)
#     # df = pd.DataFrame(data={'index': index, 'sleep_stage': y_test})
#     y_test=np.append(y_test[0],y_test)
#     if (index.shape) == (y_test.shape):
#         df = pd.DataFrame(data={'index': index, 'sleep_stage': y_test})
#         df.to_csv('model_num_'+str(fold_no)+'_our_result_GRU.csv',index=False)
#         print('you can save the results')
#     else:
#         print('there is an error in the shape of y_test')

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

def plot_heatmap(y_true,y_pred): 
    
    mat = confusion_matrix(y_true,y_pred)
    mat = mat/np.sum(mat ,axis =0)*100
    mat = mat.astype(np.int)

    fig, ax = plt.subplots()

    im, cbar = heatmap(mat, ['Wake','NREM1','NREM2','NREM3','REM'],['Wake','NREM1','NREM2','NREM3','REM'], ax=ax,
                       cmap="YlGn", cbarlabel="Pourcentage de classification")
    texts = annotate_heatmap(im, valfmt="{x:} %")

    fig.tight_layout()
    plt.show()
    
y_estimate = model1.predict([x_eeg[test], x_meta[test]])
y_estimate = np.argmax(y_estimate,axis=1)
# y_estimate=np.append(y_estimate[0],y_estimate)
plot_heatmap(y_data[test], y_estimate)

In [ ]:
# alpha = tf.constant(np.arange(2, 32*65, dtype=np.int32), shape=[32, 65, 1])
# x_out= tf.constant(np.arange(2, 32*65*100, dtype=np.int32), shape=[32, 65, 100])


In [ ]:
# init = tf.initializers.GlorotUniform()
# context_matrix = tf.Variable(init(shape=(25,50*2)),trainable=True)
# context_bias = tf.Variable(init(shape=(1,25)),trainable=True)
# context_vector = tf.Variable(init(shape=(25,1)),trainable=True)

# x = tf.random.uniform(shape=[64,65,100])
# batch_size, length, n_features = x.shape
# x_att = tf.reshape(x,[-1, n_features])
# print('x_att.shape', x_att.shape)
# u = tf.linalg.matmul(x_att,tf.transpose(context_matrix,perm=[1,0])) + context_bias
# u = tf.nn.tanh(u)
# print('u.shape',u.shape)
# uv = tf.linalg.matmul(u,context_vector)
# print('uv.shape',uv.shape)
# uv = tf.reshape(uv, [-1, length])
# print('uv.shape',uv.shape)
# alpha = tf.nn.softmax(uv,axis=1)
# print('alpha.shape',alpha.shape)
# alpha = tf.expand_dims(alpha, axis=-1)
# print('alpha.shape',alpha.shape)
# x_out = alpha * x
# print('x_out.shape',x_out.shape)
# x_out=tf.math.reduce_sum(x_out, axis=1)
# print('x_out.shape',x_out.shape)
# print(x_out)